In [688]:
# 필수(?) 라이브러리 호출
import pandas as pd
import numpy as np

In [689]:
# 경고가 많아지면 곤란해서 추가
import warnings
warnings.filterwarnings('ignore')

In [690]:
# 일단 분석할 데이터를 가져오는데
df = pd.read_csv("../raw_data/전체데이터.csv", encoding="CP949").reset_index(drop=True)
df

,회사명,거래소코드,회계년도,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결)
0,(주)CMG제약,58820,2000/12,NaN,NaN
1,(주)CMG제약,58820,2001/12,1.03,NaN
2,(주)CMG제약,58820,2002/12,-9.99,NaN
3,(주)CMG제약,58820,2003/12,-3.37,NaN
4,(주)CMG제약,58820,2004/12,-22.92,NaN
...,...,...,...,...,...
33501,흥아해운(주),3280,2015/12,0.79,1.02
33502,흥아해운(주),3280,2016/12,0.04,0.21
33503,흥아해운(주),3280,2017/12,-0.61,-0.42
33504,흥아해운(주),3280,2018/12,-1.26,-1.16


In [691]:
# 분석 과정에서 NaN이 NaN이 아닌 다른 어떤 값인 것으로 판명되었습니다.
# 어쩔 수 없이 분석의 용이함을 위해 임의의 값을 채웠습니다
df = df.fillna(77777.77)

# 이건 우리의 기준에 맞는 이자보상 배율을 저장하기 위한 컬럼입니다.
df['이자보상배율'] = 0.0

In [692]:
# csv 한바퀴 돌립니다
for i in range(0,len(df)-1):
    #print(i)

    # 구 기준 회계값에 결측치가 없다면
    if df.iloc[i]['이자보상배율(이자비용)'] != 77777.77:
        # 신 기준 회계값에도 결측치가 없다면
        if df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'] != 77777.77:
            # 신 회계값 삽입
            df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']
            #print(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'])

        # 신 회계가 결측치라면
        else:
            df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)'] # 구 회계값 삽입
            #print(df.iloc[i]['이자보상배율(이자비용)'])

    # 구 기준 회계값에 결측치가 있다면
    else:
        # 신 회계 기준 회계값에 결측치가 없다면
        if df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'] != 77777.77:
            # 신 회계값 삽입
            df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']
            #print(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'])

        # 이자보상배율에 대한 데이터가 아예 없는 경우 임의의 값 삽입
        else:
            df.loc[i, "이자보상배율"] = 11111111.11

In [693]:
df

,회사명,거래소코드,회계년도,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),이자보상배율
0,(주)CMG제약,58820,2000/12,77777.77,77777.77,11111111.11
1,(주)CMG제약,58820,2001/12,1.03,77777.77,1.03
2,(주)CMG제약,58820,2002/12,-9.99,77777.77,-9.99
3,(주)CMG제약,58820,2003/12,-3.37,77777.77,-3.37
4,(주)CMG제약,58820,2004/12,-22.92,77777.77,-22.92
...,...,...,...,...,...,...
33501,흥아해운(주),3280,2015/12,0.79,1.02,1.02
33502,흥아해운(주),3280,2016/12,0.04,0.21,0.21
33503,흥아해운(주),3280,2017/12,-0.61,-0.42,-0.42
33504,흥아해운(주),3280,2018/12,-1.26,-1.16,-1.16


In [694]:
df['이자보상배율'].value_counts()

 1.000000e+09    1773
 1.111111e+07    1385
 1.300000e+00      49
 1.110000e+00      43
 1.950000e+00      42
                 ... 
 1.128690e+03       1
 7.726600e+02       1
-1.126300e+02       1
-1.755000e+01       1
 2.520200e+02       1
Name: 이자보상배율, Length: 9816, dtype: int64

In [695]:
# 우리가 앞으로 작업을 엄청나게 복잡하게 할 걸 생각하면
# 중간중간 결과들을 데이터프레임으로 만들고 빼주는 게 좋습니다
df_2 = df[['회사명', '거래소코드', '회계년도', '이자보상배율']]
df_2

,회사명,거래소코드,회계년도,이자보상배율
0,(주)CMG제약,58820,2000/12,11111111.11
1,(주)CMG제약,58820,2001/12,1.03
2,(주)CMG제약,58820,2002/12,-9.99
3,(주)CMG제약,58820,2003/12,-3.37
4,(주)CMG제약,58820,2004/12,-22.92
...,...,...,...,...
33501,흥아해운(주),3280,2015/12,1.02
33502,흥아해운(주),3280,2016/12,0.21
33503,흥아해운(주),3280,2017/12,-0.42
33504,흥아해운(주),3280,2018/12,-1.16


In [696]:
# 일단 우리의 소중한 결과물을 빼놓도록 하죠
df_2.to_csv("../csv_data/이자보상배율1차.csv", encoding="CP949")

In [697]:
df_2

,회사명,거래소코드,회계년도,이자보상배율
0,(주)CMG제약,58820,2000/12,11111111.11
1,(주)CMG제약,58820,2001/12,1.03
2,(주)CMG제약,58820,2002/12,-9.99
3,(주)CMG제약,58820,2003/12,-3.37
4,(주)CMG제약,58820,2004/12,-22.92
...,...,...,...,...
33501,흥아해운(주),3280,2015/12,1.02
33502,흥아해운(주),3280,2016/12,0.21
33503,흥아해운(주),3280,2017/12,-0.42
33504,흥아해운(주),3280,2018/12,-1.16


In [698]:
# 현재 데이터 기준으로 한계기업과 4년 연속 이.보.배 1 미만인 기업을 찾을 겁니다

f_company = [] # 한계기업을 저장하는 리스트
f_company_x = [] # 4년 연속 이자보상배율 1 미만인 기업 삽입
count = 0 # 한계기업 카운트용
company = 'x' # 한계기업 기록용

# 이거도 csv 한바퀴 돌립니다
for i in range(0, len(df_2)-1):

    # 회사명이 이미 기록된 상태에서 또 중복으로 적을 필요는 없습니다
    if company == df.iloc[i]['회사명']:
        # 다만 그 기업이 한계기업인데
        if count > 3:
            # 어머? 4년 연속이네요? 기록!!!
            if df.iloc[i]['이자보상배율'] < 1:
                f_company_x.append(company)
                count = 0

                # 반복문에서 볼일 다 끝났으니 통과합니다
                continue
        
        # 그게 아니라면 반복문을 통과합니다
        continue

    # 이자보상배율이 1 미만인 경우 카운트를 늘립니다
    if df.iloc[i]['이자보상배율'] < 1:
            count += 1
    # 도중에 이자보상배율이 1 미만이면 탈출합니다
    else:
        count = 0

    # 한계 기업 클럽에 오셨습니다. 환영합니다.
    if count == 3:
        company = df.iloc[i]['회사명']
        f_company.append(company)
        count += 1

In [699]:
f_company

['(주)CMG제약',
 '(주)DB하이텍',
 '(주)ES큐브',
 '(주)HJ중공업',
 '(주)SH에너지화학',
 '(주)STX',
 '(주)TCC스틸',
 '(주)강스템바이오텍',
 '(주)강원에너지',
 '(주)갤럭시아에스엠',
 '(주)경방',
 '(주)골드앤에스',
 '(주)골드퍼시픽',
 '(주)광림',
 '(주)광무',
 '(주)국동',
 '(주)국보',
 '(주)국순당',
 '(주)그래디언트',
 '(주)금양',
 '(주)기가레인',
 '(주)기산텔레콤',
 '(주)까뮤이앤씨',
 '(주)나노',
 '(주)나노브릭',
 '(주)나노엔텍',
 '(주)나이벡',
 '(주)남선알미늄',
 '(주)남성',
 '(주)내츄럴엔도텍',
 '(주)네오리진',
 '(주)네이처셀',
 '(주)네패스',
 '(주)넥스트엔터테인먼트월드',
 '(주)넥슨게임즈',
 '(주)넵튠',
 '(주)노루홀딩스',
 '(주)노블엠앤비',
 '(주)녹십자',
 '(주)녹십자엠에스',
 '(주)녹원씨엔아이',
 '(주)누리플랜',
 '(주)누리플렉스',
 '(주)뉴로스',
 '(주)뉴보텍',
 '(주)뉴인텍',
 '(주)뉴프렉스',
 '(주)다날',
 '(주)다산네트웍스',
 '(주)다우기술',
 '(주)다이나믹디자인',
 '(주)대구백화점',
 '(주)대명소노시즌',
 '(주)대성엘텍',
 '(주)대양금속',
 '(주)대유에이텍',
 '(주)대유플러스',
 '(주)대창',
 '(주)대창솔루션',
 '(주)대한항공',
 '(주)대호에이엘',
 '(주)대호특수강',
 '(주)더 미동',
 '(주)더블유아이',
 '(주)더존비즈온',
 '(주)동방선기',
 '(주)동양',
 '(주)동운아나텍',
 '(주)두산',
 '(주)두올',
 '(주)듀오백',
 '(주)드래곤플라이',
 '(주)드림어스컴퍼니',
 '(주)디모아',
 '(주)디비아이엔씨',
 '(주)디아크',
 '(주)디에스앤엘',
 '(주)디엔에이링크',
 '(주)디엠에스',
 '(주)디지아이',
 '(주)디지틀조선일보',
 

In [700]:
f_company_x

['(주)CMG제약',
 '(주)DB하이텍',
 '(주)ES큐브',
 '(주)HJ중공업',
 '(주)SH에너지화학',
 '(주)STX',
 '(주)강스템바이오텍',
 '(주)강원에너지',
 '(주)갤럭시아에스엠',
 '(주)경방',
 '(주)골드앤에스',
 '(주)골드퍼시픽',
 '(주)광림',
 '(주)광무',
 '(주)국동',
 '(주)국보',
 '(주)국순당',
 '(주)그래디언트',
 '(주)금양',
 '(주)기가레인',
 '(주)기산텔레콤',
 '(주)까뮤이앤씨',
 '(주)나노',
 '(주)나노브릭',
 '(주)나노엔텍',
 '(주)나이벡',
 '(주)남선알미늄',
 '(주)남성',
 '(주)내츄럴엔도텍',
 '(주)네오리진',
 '(주)네이처셀',
 '(주)넵튠',
 '(주)노블엠앤비',
 '(주)녹십자엠에스',
 '(주)녹원씨엔아이',
 '(주)누리플랜',
 '(주)뉴보텍',
 '(주)뉴인텍',
 '(주)뉴프렉스',
 '(주)다날',
 '(주)다산네트웍스',
 '(주)다우기술',
 '(주)대구백화점',
 '(주)대명소노시즌',
 '(주)대성엘텍',
 '(주)대양금속',
 '(주)대유에이텍',
 '(주)대유플러스',
 '(주)대창',
 '(주)대창솔루션',
 '(주)대한항공',
 '(주)대호에이엘',
 '(주)더 미동',
 '(주)더블유아이',
 '(주)더존비즈온',
 '(주)동방선기',
 '(주)동양',
 '(주)두산',
 '(주)두올',
 '(주)듀오백',
 '(주)드래곤플라이',
 '(주)드림어스컴퍼니',
 '(주)디모아',
 '(주)디비아이엔씨',
 '(주)디아크',
 '(주)디에스앤엘',
 '(주)디엔에이링크',
 '(주)디엠에스',
 '(주)디지아이',
 '(주)디지틀조선일보',
 '(주)라온테크',
 '(주)라파스',
 '(주)레고켐바이오사이언스',
 '(주)레드로버',
 '(주)레드캡투어',
 '(주)룽투코리아',
 '(주)마니커',
 '(주)매직마이크로',
 '(주)머큐리',
 '(주)메디콕스',
 '(주)메지온',
 '(주)메타랩

In [701]:
# 분명히 이거보다 더 좋은 방법이 있을건데
# 맷돌이 굴러가질 않네요

# 이건 f_company 리스트에 들어잇는 데이터와 일치하면 True, 아니면 False를 반환합니다
df_4 = df_2.isin(f_company)
df_4

,회사명,거래소코드,회계년도,이자보상배율
0,True,False,False,False
1,True,False,False,False
2,True,False,False,False
3,True,False,False,False
4,True,False,False,False
...,...,...,...,...
33501,True,False,False,False
33502,True,False,False,False
33503,True,False,False,False
33504,True,False,False,False


In [702]:
df_5 = [] # 임시 저장용 리스트

# 그렇게 만든 데이터프레임을 기존과 비교해서 True인 애들만 뽑아옵니다
for i in range(0, len(df_4)-1):
    if df_4.iloc[i]["회사명"] == True:
        df_5.append(df_2.loc[i])

In [703]:
# 그리고 저장을 합니다
df_5_frame = pd.DataFrame(df_5).reset_index(drop=True)
df_5_frame.to_csv("../csv_data/한계기업후보.csv", encoding="CP949")

In [704]:
# 4년 연속 이자 보상 배율이 1 미만인 기업들도 똑같이 해봅니다

df_6 = df_2.isin(f_company_x)
df_7 = []

for i in range(0, len(df_6)-1):
    if df_6.iloc[i]["회사명"] == True:
        df_7.append(df_2.loc[i])

df_7_frame = pd.DataFrame(df_7).reset_index(drop=True)

In [705]:
# 중간중간 저장은 선택이 아닌 필수
df_7_frame.to_csv("../csv_data/회생불가1차.csv", encoding="CP949")

In [706]:
# 그래서 4년 연속인 애들이 얼마나 될까요? 밑에 숫자를 봅시다
df_7_frame.copy().drop_duplicates(['회사명'])

,회사명,거래소코드,회계년도,이자보상배율
0,(주)CMG제약,58820,2000/12,11111111.11
20,(주)DB하이텍,990,2000/12,1.01
40,(주)ES큐브,50120,2001/06,11111111.11
59,(주)HJ중공업,97230,2007/12,11111111.11
72,(주)SH에너지화학,2360,2000/06,0.76
...,...,...,...,...
11574,한화시스템(주),272210,2000/12,11111111.11
11594,현대바이오사이언스(주),48410,2000/12,11111111.11
11614,현대에이치티(주),39010,2000/12,9843.76
11634,휴림로봇(주),90710,2003/12,11111111.11


In [707]:
# 한계기업 후보 기업의 숫자는 밑에 카운트를 봅시다
df_5_frame.copy().drop_duplicates(['회사명'])

,회사명,거래소코드,회계년도,이자보상배율
0,(주)CMG제약,58820,2000/12,11111111.11
20,(주)DB하이텍,990,2000/12,1.01
40,(주)ES큐브,50120,2001/06,11111111.11
59,(주)HJ중공업,97230,2007/12,11111111.11
72,(주)SH에너지화학,2360,2000/06,0.76
...,...,...,...,...
13505,현대에이치티(주),39010,2000/12,9843.76
13525,현우산업(주),92300,2001/12,11111111.11
13544,화성산업(주),2460,2000/12,1.26
13564,휴림로봇(주),90710,2003/12,11111111.11
